#  Faster HuggingFace🤗 Transformers Training
> with dynamic padding and order batches
- toc: true
- branch: master
- badges: true
- image: images/dynamic-pad.png
- comments: true
- author: Sajjad Ayoubi
- categories: [tips]

- dynamic padding at one glance

![](https://uupload.ir/files/79ki_dynamic-pad.png)

In [ ]:
!nvidia-smi

Sat Mar 13 03:21:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

- installing normal things

In [ ]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 1.9MB 18.2MB/s 
     |████████████████████████████████| 3.2MB 58.2MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
     |████████████████████████████████| 194kB 18.2MB/s 
     |████████████████████████████████| 112kB 13.9MB/s 
     |████████████████████████████████| 245kB 25.1MB/s 


- importing normal things

In [ ]:
import random, os
import numpy as np
from copy import deepcopy

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

- set all params that we need

In [ ]:
model_checkpoint = "distilbert-base-uncased"
max_length = 512
batch_size = 16
epochs = 2
lr = 2e-5

## loading IMBD dataset 
- as an example

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
train, test = load_dataset('imdb', split=['train', 'test'])

In [ ]:
toknize_with_padding = lambda x: tokenizer(x['text'], truncation=True, padding=True)

# prepare datasets
train_ds = train.map(toknize_with_padding, batched=True, batch_size=128, remove_columns=['text']).shuffle()
test_ds = test.map(toknize_with_padding, batched=True, batch_size=128, remove_columns=['text']).shuffle()

In [ ]:
def torch_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)

torch_seed()

- loading model and saveing original weights for reproducibility

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
weights = deepcopy(model.state_dict())

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

## Normal Padding
- train with padding until biggest example in dataset
- or set a max_len 

In [ ]:
args = TrainingArguments(
    f"result",
    evaluation_strategy='epoch',
    save_steps=999999, # we don't need saving 
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    )

In [ ]:
model.load_state_dict(weights)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    )

trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.227000,0.206788,469.372300,53.263000
2,0.124700,0.236525,470.060200,53.185000


TrainOutput(global_step=3126, training_loss=0.20294074926785147, metrics={'train_runtime': 3544.891, 'train_samples_per_second': 0.882, 'total_flos': 10284289536000000, 'epoch': 2.0})

- it takes about one hour right

## Dynamic Padding


- first of all we don't need padding in tokenization stage
  - we doing it on training time

In [ ]:
toknize_wo_padding = lambda x: tokenizer(x['text'], truncation=True, padding=False)

# prepare datasets
train_ds = train.map(toknize_wo_padding, batched=True, batch_size=128, remove_columns=['text'])
test_ds = test.map(toknize_wo_padding, batched=True, batch_size=128, remove_columns=['text'])

- we need a pytorch data collator which pad each batch until biggest example on it

In [ ]:
# add padding until to biggest seq in this batch
def add_pad(seq, max_batch_len, pad_token):
    return seq + (max_batch_len - len(seq)) * [pad_token]

# PyTorch data collator
def dynamic_padding(batch):
    batch_input_ids = []
    batch_attention_masks = []
    labels = []
    max_size = max([len(i['input_ids']) for i in batch])
    for i in batch:
        batch_input_ids.append(add_pad(i['input_ids'], max_size, tokenizer.pad_token_id)) # add pad for ids
        batch_attention_masks.append(add_pad(i['attention_mask'], max_size, 0)) # add pad for mask
        labels.append(i['label'])

    return {"input_ids": torch.tensor(batch_input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)}

In [ ]:
model.load_state_dict(weights)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=dynamic_padding # passing dynamic padding as a new collator
    )

trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.215100,0.193075,458.983600,54.468000
2,0.131700,0.240500,459.432200,54.415000


TrainOutput(global_step=3126, training_loss=0.2033301882460113, metrics={'train_runtime': 3524.2944, 'train_samples_per_second': 0.887, 'total_flos': 10190053305445440, 'epoch': 2.0})

- with just dynamic padding we have not a huge improvement
  - it depends on the variety of sequence length in the dataset
  - on the SQuAD2 dataset, I Have x1.5 boost just with Dynamic Padding

## Dynamic Padding With Sorting

In [ ]:
class SortedDataset(Dataset):
  def __init__(self, examples):
    self.examples = sorted(examples, key=lambda t: len(t['input_ids']))

  def __getitem__(self, i):
    return self.examples[i]

  def __len__(self):
    return len(self.examples)

sorted_train_ds = SortedDataset(train_ds.shuffle())
sorted_test_ds = SortedDataset(test_ds.shuffle())

- in transformer.Trainer default sampler is RandomSampler
  - we don't wanna shuffle data
  - we ovverride Trainer class and create a new DataLoder with SequentialSampler

In [ ]:
from torch.utils.data.sampler import SequentialSampler

# new Trainer class with SequentialSampler
class SortedTrainer(Trainer):
  def get_train_dataloader(self):    
    return DataLoader(
        self.train_dataset,
        batch_size=self.args.train_batch_size,
        sampler=SequentialSampler(self.train_dataset), # for don't doing shuffle on train_ds
        collate_fn=self.data_collator)

In [ ]:
model.load_state_dict(weights)

In [ ]:
trainer = SortedTrainer( # using SortedTrainer instead of Trainer
    model=model,
    args=args,
    train_dataset=sorted_train_ds,
    eval_dataset=sorted_test_ds,
    data_collator=dynamic_padding)

trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.244000,0.192687,241.501000,103.519000
2,0.143500,0.256823,242.278800,103.187000


TrainOutput(global_step=3126, training_loss=0.2028842329292517, metrics={'train_runtime': 1925.2676, 'train_samples_per_second': 1.624, 'total_flos': 5532806361386880, 'epoch': 2.0})

- now with sorted dataset we have 2 time faster training
  - but we have not the advantage of stochastic training
  - the simplest solution is Sortish Training

## Dynamic Padding with Sortish

- sortish mean sort each mega batch instead of entire dataset
  - for having the advantage of randomness with padding

In [ ]:
class SortishDataset(Dataset):
  def __init__(self, examples, batch_size=16, orderish=True, mega_size=30):
    n = len(examples)
    if orderish:  step = batch_size*mega_size
    else: step = int(batch_size*(1/batch_size))
    
    sortish_examples = []
    for mega_batch in range(0, n, step):
      if mega_batch+step>n:
          sortish_examples += sorted(examples.select(range(mega_batch, n)), key=lambda t: len(t['input_ids']))
          break
      sortish_examples += sorted(examples.select(range(mega_batch, mega_batch+step)), key=lambda t: len(t['input_ids']))

    self.sortish_examples = sortish_examples

  def __getitem__(self, i):
    return self.sortish_examples[i]

  def __len__(self):
    return len(self.sortish_examples)


sortish_train_ds = SortishDataset(train_ds.shuffle(), orderish=True, batch_size=batch_size)
sortish_test_ds = SortishDataset(test_ds, orderish=False, batch_size=batch_size)

In [ ]:
model.load_state_dict(weights)

In [ ]:
trainer = SortedTrainer(
    model=model,
    args=args,
    train_dataset=sortish_train_ds,
    eval_dataset=sortish_test_ds,
    data_collator=dynamic_padding)

trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.225600,0.228117,459.673200,54.386000
2,0.131000,0.231613,459.862300,54.364000


- not bad ha??
  - 40 mins without any effects on training